In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [ ]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

# skills
# remove "XP:" from result
# some abilities need additional cleaning

In [2]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [3]:
##########################################Get soup request#########################################################
def get_soup(url):
    '''Child of get_skills_df
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup.find(id='list')

In [22]:
def clean_ability_string(ability):
    
    # replace icon html with matching word in all caps
    icon_types = [
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        ability = ability.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        ability = ability.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    ability = ability.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    ability = ability.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>',]
    
    for item in dirt:
        
        ability = ability.replace(item,'')

    return ability

In [4]:
def get_stat_line(results):
    
    stat_line = results.find('div', class_='card-props').text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1], stat_list[2], stat_list[3]


def get_stam_line(results):
    
    stat_line = results.find('div', class_='card-props').find_next().text.replace('\n', '').replace('\t', '')

    stat_list = [int(char) for char in str(stat_line) if char.isnumeric() == True]

    return stat_list[0], stat_list[1]


def get_deck_reqs(results, faction):
    
    deck = results.find_all('div', class_=f'card-text border-{faction.lower()}')[1].text

    deck_split = re.split(':', deck)

    deck_size = re.search('[0-9]{2}', deck_split[1]).group()
    
    deck_options = re.search('^(.*?\.)', deck_split[2]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')

    must_include = re.search('^(.*?\.)', deck_split[3]).group()[1:-1].replace('(', '').replace(')', '').replace('  ', ' ')
    
    return deck_size, deck_options, must_include

In [14]:
def get_invest_traits(results):
    '''Scrapes and returns investigator card data'''

    title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

    faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

    tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

    traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

    willpower, intellect, combat, agility = get_stat_line(results)

    health, sanity = get_stam_line(results)
   
    ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

    artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

    expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

    try:
        
        flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

    except:
        
        flavor = ''
        
    deck_size, deck_options, must_include = get_deck_reqs(results, faction)
    
    return [title, 
            faction, 
            tipe, 
            traits, 
            willpower, 
            intellect, 
            combat, 
            agility, 
            health, 
            sanity,
            ability, 
            artist,
            expansion,
            flavor,
            deck_size,
            deck_options,
            must_include]

In [15]:
# make empty df

def get_invest_df():

    investigator_traits = { 'title':[],
                            'faction':[],
                            'type':[],
                            'traits':[],                      
                            'willpower':[],
                            'intellect':[],
                            'combat':[],
                            'agility':[],                       
                            'health':[],
                            'sanity':[],                     
                            'ability':[],
                            'artist':[],
                            'expansion':[],
                            'flavor':[],
                            'deck_size':[],
                            'deck_options':[],
                            'must_include':[]}

    # for each url get player card info from that page and add each element to skill_traits
    for url in invest:

        # make html request to arkham db and parse using BS
        results = get_soup(url)

        # get list of card elements card elements
        trait_list = get_invest_traits(results)

        print(f'Getting investigator card {trait_list[0]}...')

        # itterate through card element titles and add each to a dictionary
        for i, key in enumerate(investigator_traits):

            investigator_traits[key].append(trait_list[i])

    print("Making dataframe...")

    return pd.DataFrame(investigator_traits)

Getting investigator card "Ashcan" Pete...
Getting investigator card "Skids" O'Toole...
Getting investigator card "Skids" O'Toole...
Getting investigator card Agnes Baker...
Getting investigator card Agnes Baker...
Getting investigator card Akachi Onyele...
Getting investigator card Alessandra Zorzi...
Getting investigator card Amanda Sharpe...
Getting investigator card Amina Zidane...
Getting investigator card Bob Jenkins...
Getting investigator card Calvin Wright...
Getting investigator card Carolyn Fern...
Getting investigator card Carolyn Fern...
Getting investigator card Carson Sinclair...
Getting investigator card Charlie Kane...
Getting investigator card Daisy Walker...
Getting investigator card Daisy Walker...
Getting investigator card Daniela Reyes...
Getting investigator card Darrell Simmons...
Getting investigator card Dexter Drake...
Getting investigator card Diana Stanley...
Getting investigator card Father Mateo...
Getting investigator card Finn Edwards...
Getting investi

In [16]:
df_invest.head(10)

,title,faction,type,traits,willpower,intellect,combat,agility,health,sanity,ability,artist,expansion,flavor,deck_size,deck_options,must_include
0,"""Ashcan"" Pete",Survivor,Investigator,Drifter.,4,2,2,3,6,5,You begin the game with Duke in play. Discard ...,Jake Murray,The Dunwich Legacy #5,"At night, ""Ashcan"" Pete sleeps wherever his tr...",30,"Survivor cards level 0-5, Neutral cards level ...","Duke, Wracked by Nightmares, 1 random basic we..."
1,"""Skids"" O'Toole",Rogue,Investigator,Criminal.,2,3,3,4,8,6,"During your turn, spend 2 resources: You may ...",Magali Villeneuve,Core Set #3,Skids hadn't planned on a life of crime. But s...,30,"Rogue cards level 0-5, Guardian cards level 0-...","On the Lam, Hospital Debts, 1 random basic wea..."
2,"""Skids"" O'Toole",Rogue,Investigator,Criminal.,2,3,3,4,8,6,"During your turn, spend 2 resources: You may ...",Magali Villeneuve,Revised Core Set #3,Skids hadn't planned on a life of crime. But s...,30,"Rogue cards level 0-5, Guardian cards level 0-...","On the Lam, Hospital Debts, 1 random basic wea..."
3,Agnes Baker,Mystic,Investigator,Sorcerer.,5,2,2,3,6,8,After 1 or more horror is placed on Agnes Bak...,Magali Villeneuve,Core Set #4,Agnes Baker may be just an unassuming waitress...,30,"Mystic cards level 0-5, Survivor cards level 0...","Heirloom of Hyperborea, Dark Memory, 1 random ..."
4,Agnes Baker,Mystic,Investigator,Sorcerer.,5,2,2,3,6,8,After 1 or more horror is placed on Agnes Bak...,Magali Villeneuve,Revised Core Set #4,Agnes Baker may be just an unassuming waitress...,30,"Mystic cards level 0-5, Survivor cards level 0...","Heirloom of Hyperborea, Dark Memory, 1 random ..."
5,Akachi Onyele,Mystic,Investigator,Sorcerer.,5,2,3,3,6,8,"Your assets with ""uses (charges)"" enter play w...",Magali Villeneuve,The Path to Carcosa #4,"As a young girl in Nigeria, Akachi became used...",30,"Mystic cards level 0-5, Neutral cards level 0-...","Spirit-Speaker, Angered Spirits, 1 random basi..."
6,Alessandra Zorzi,Rogue,Investigator,Drifter. Socialite.,3,4,2,4,7,7,You may take an additional action during your ...,Aleksander Karcz,The Feast of Hemlock Vale Investigator Expansi...,Alessandra's life of carefree luxury enables h...,30,"Rogue cards level 0-5, cards with ""parley"" lev...","3 copies of Beguile, Zamacona, 1 random basic ..."
7,Amanda Sharpe,Seeker,Investigator,Miskatonic. Scholar.,2,2,2,2,7,7,Forced - At the start of the investigation pha...,Cristi Balanescu,The Innsmouth Conspiracy #2,Amanda Sharpe was on track to become one of Mi...,30,"Seeker cards level 0-5, Neutral cards level 0-...","Obscure Studies, Whispers from the Deep, 1 ran..."
8,Amina Zidane,Mystic,Investigator,Chosen. Cursed.,3,3,3,3,5,9,When you play an asset: Reduce its resource c...,Aleksander Karcz,The Scarlet Keys Investigator Expansion #11,"A refugee from French-occupied Algeria, Amina ...",30,"Mystic cards level 0-5, Neutral cards level 0-...","Word of Woe, Word of Weal, Deafening Silence, ..."
9,Bob Jenkins,Survivor,Investigator,Entrepreneur.,2,4,3,3,6,8,"At any time, an investigator at your location ...",Cristi Balanescu,Edge of the Earth Investigator Expansion #16,"According to Bob, the secret to success as a s...",30,"Survivor cards level 0, Rogue cards level 1-5,...","Shrewd Dealings, Greed, 1 random basic weakness"


In [ ]:
# [title, test_icons, traits, willpower, intellect, combat, agility, health, sanity, faction, ability, tipe, flavor, artist, expansion]

In [17]:
# Investigator Traits

results = get_soup(assets[20])

title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

willpower, intellect, combat, agility = get_stat_line(results)

health, sanity = get_stam_line(results)

ability = results.find('div', class_=f'card-text border-{faction.lower()}').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '') 

flavor = results.find_all('div', class_='card-flavor small')[1].text.replace('\n', '').replace('\t', '')

deck_size, deck_options, must_include = get_deck_reqs(results)

TypeError: get_deck_reqs() missing 1 required positional argument: 'faction'

In [24]:
str(results.find('div', class_=f'card-text border-{faction.lower()}'))

'<div class="card-text border-mystic">\n<p>Diana Stanley gets +1 <span class="icon-willpower" title="Willpower"></span> for each card beneath her.</p><p><span class="icon-reaction" title="Reaction"></span> After a card you own cancels or ignores a card effect or game effect, if there are fewer than 5 cards beneath Diana Stanley: Place that card facedown beneath her. Draw 1 card and gain 1 resource. (Limit once per phase.)</p><p><span class="icon-elder_sign" title="Elder Sign"></span> effect: +2. You may choose a card beneath Diana Stanley and add it to your hand.</p>\n</div>'

In [23]:
dirty_string = str(results.find('div', class_=f'card-text border-{faction.lower()}'))
clean_ability_string(dirty_string)

'Diana Stanley gets +1 WILLPOWER for each card beneath her.REACTION After a card you own cancels or ignores a card effect or game effect, if there are fewer than 5 cards beneath Diana Stanley: Place that card facedown beneath her. Draw 1 card and gain 1 resource. (Limit once per phase.)ELDER_SIGN effect: +2. You may choose a card beneath Diana Stanley and add it to your hand.'

In [18]:
get_soup(assets[20])

<div id="list">
<div class="row">
<div class="col-sm-12">
<ul class="pager">
<li class="previous">
<a class="no-popup" href="/card/05039">← Five of Pentacles</a> </li>
<li>
<a href="/set/tcu">The Circle Undone</a>
</li>
<li class="next">
<a class="no-popup" href="/card/05041">The 13th Vision →</a> </li>
</ul>
</div>
</div>
<div class="row">
<div class="col-md-9">
<div class="row">
<div class="col-sm-7">
<div>
<div class="panel panel-default border-neutral">
<div class="panel-heading border-neutral bg-neutral">
<h3 class="panel-title">
<a class="card-name card-tip" data-code="05040" href="https://arkhamdb.com/card/05040">Ace of Rods
</a>
<div class="card-subname small">The Fateful Step</div> </h3>
</div>
<div class="panel-body card-content">
<div class="card-faction">
<span class="icon-neutral fg-neutral"></span>
<span class="card-faction">Neutral</span>
</div>
<div class="card-info">
<span class="card-type">Asset. </span>
<p class="card-traits">Tarot.</p>
<div class="card-props">
		Cos

In [ ]:
title

In [ ]:
faction

In [ ]:
tipe

In [ ]:
traits

In [ ]:
willpower

In [ ]:
intellect

In [ ]:
combat

In [ ]:
agility

In [ ]:
health

In [ ]:
sanity

In [ ]:
ability

In [ ]:
artist

In [ ]:
expansion

In [ ]:
deck_size

In [ ]:
flavor

In [ ]:
deck_size

In [ ]:
deck_options

In [ ]:
must_include 